## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-19-05-42-16 +0000,nyt,White House Meeting Ends With Few Signs of Pro...,https://www.nytimes.com/live/2025/08/18/us/tru...
1,2025-08-19-05-38-01 +0000,nyt,Zelensky Says He’s Ready to Meet Putin After T...,https://www.nytimes.com/2025/08/18/us/zelensky...
2,2025-08-19-05-37-10 +0000,bbc,Watch: Zelensky's 'fabulous' suit and other mo...,https://www.bbc.com/news/videos/crlzey2j406o?a...
3,2025-08-19-05-35-40 +0000,nyt,"Melania Trump Sends a Letter to Putin, Asking ...",https://www.nytimes.com/2025/08/18/us/melania-...
4,2025-08-19-05-28-13 +0000,nypost,Illegal migrant caught working as police offic...,https://nypost.com/2025/08/19/us-news/illegal-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,51
8,zelensky,28
7,ukraine,23
1,house,18
436,will,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
219,2025-08-18-11-37-54 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...,155
144,2025-08-18-18-02-57 +0000,nypost,Trump ‘very happy’ after Ukraine peace summit ...,https://nypost.com/2025/08/18/us-news/trump-ze...,148
228,2025-08-18-10-27-12 +0000,nypost,Harrowing photo shows teen killed in Russian s...,https://nypost.com/2025/08/18/world-news/russi...,146
17,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...,144
217,2025-08-18-12-03-49 +0000,wapo,"What to expect as Zelensky, European leaders m...",https://www.washingtonpost.com/world/2025/08/1...,130


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
219,155,2025-08-18-11-37-54 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...
117,66,2025-08-18-19-54-53 +0000,nypost,Putin claims Ukraine should give up Donetsk be...,https://nypost.com/2025/08/18/world-news/putin...
199,50,2025-08-18-13-53-28 +0000,cbc,Air Canada flight attendants' union says it wi...,https://www.cbc.ca/news/politics/air-canada-st...
46,41,2025-08-19-00-22-32 +0000,nypost,Furious Harvard researcher claims he was pumme...,https://nypost.com/2025/08/18/us-news/furious-...
12,39,2025-08-19-03-51-58 +0000,nyt,European Leaders’ Rush to Trump-Zelensky Meeti...,https://www.nytimes.com/2025/08/18/world/europ...
119,35,2025-08-18-19-47-16 +0000,nyt,Have You Been Unemployed for Six Months or Mor...,https://www.nytimes.com/2025/08/18/business/lo...
69,35,2025-08-18-23-13-00 +0000,wsj,Texas Democrats ended a two-week walkout over ...,https://www.wsj.com/politics/policy/texas-demo...
82,34,2025-08-18-22-20-19 +0000,nypost,MSNBC gets mercilessly mocked over new MS NOW ...,https://nypost.com/2025/08/18/media/msnbc-gets...
135,33,2025-08-18-18-32-30 +0000,latimes,Newsmax to pay $67 million to settle Dominion ...,https://www.latimes.com/entertainment-arts/bus...
78,31,2025-08-18-22-44-21 +0000,latimes,'Ketamine Queen' to plead guilty to federal ch...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
